## Imports

In [1]:
import os

from mmaction.datasets import build_dataset, build_dataloader
from mmaction.models import build_model
from mmcv import Config
from mmaction.datasets import MixupBlending
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torch
import logging
import random
import numpy as np
import torch.nn.functional as F
import csv
import pandas as pd

/home/sadat/miniconda3/envs/scrambmix/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/sadat/miniconda3/envs/scrambmix/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/home/sadat/miniconda3/envs/scrambmix/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.17.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def precision_score(y_true, y_pred):
    """Calculates precision score.

    Args:
        y_true: True labels.
        y_pred: Predicted labels.

    Returns:
        Precision score.
    """

    tp = sum(y_true[i] == y_pred[i] for i in range(len(y_true)) if y_pred[i] == 1)
    fp = sum(y_true[i] != y_pred[i] for i in range(len(y_true)) if y_pred[i] == 1)
    precision = tp / (tp + fp) if tp + fp != 0 else 0
    return precision

def recall_score(y_true, y_pred):
    """Calculates recall score.

    Args:
        y_true: True labels.
        y_pred: Predicted labels.

    Returns:
        Recall score.
    """

    tp = sum(y_true[i] == y_pred[i] for i in range(len(y_true)) if y_pred[i] == 1)
    fn = sum(y_true[i] == 1 and y_pred[i] != 1 for i in range(len(y_true)))
    recall = tp / (tp + fn) if tp + fn != 0 else 0
    return recall

def f1_score(y_true, y_pred):
    """Calculates F1 score.

    Args:
        y_true: True labels.
        y_pred: Predicted labels.

    Returns:
        F1 score.
    """

    p = precision_score(y_true, y_pred)
    r = recall_score(y_true, y_pred)
    return 2 * p * r / (p + r) if p + r != 0 else 0

def weighted_f1_score(y_true, y_pred):
    """Calculates the weighted F1 score, assuming equal class weights.

    Args:
        y_true: True labels.
        y_pred: Predicted labels.

    Returns:
        Weighted F1 score.
    """

    num_classes = len(set(y_true))
    f1_scores = []
    for i in range(num_classes):
        class_mask = [1 if y == i else 0 for y in y_true]
        class_f1 = f1_score(class_mask, [1 if y == i else 0 for y in y_pred])
        f1_scores.append(class_f1)
    return sum(f1_scores) / num_classes

def accuracy_score(y_true, y_pred):
    """Calculates the accuracy score.
    
    Args:
    y_true: True labels.
    y_pred: Predicted labels.
    
    Returns:
    Accuracy score.
    """
    
    correct_predictions = sum(np.array(y_true) == np.array(y_pred))
    total_predictions = len(y_true)
    accuracy = correct_predictions / total_predictions

    return accuracy

## Loading batches

In [3]:
cfg = Config.fromfile('./mixup.py')

In [4]:
os.chdir('../../..')

In [5]:
cfg.data.train

{'type': 'RawframeDataset',
 'ann_file': 'data/hmdb51/annotation_train.txt',
 'data_prefix': 'data/hmdb51/rawframes',
 'pipeline': [{'type': 'SampleFrames',
   'clip_len': 32,
   'frame_interval': 2,
   'num_clips': 1},
  {'type': 'RawFrameDecode'},
  {'type': 'Resize', 'scale': (-1, 256)},
  {'type': 'RandomResizedCrop'},
  {'type': 'Resize', 'scale': (224, 224), 'keep_ratio': False},
  {'type': 'Flip', 'flip_ratio': 0.5},
  {'type': 'Normalize',
   'mean': [123.675, 116.28, 103.53],
   'std': [58.395, 57.12, 57.375],
   'to_bgr': False},
  {'type': 'FormatShape', 'input_format': 'NCTHW'},
  {'type': 'Collect', 'keys': ['imgs', 'label'], 'meta_keys': []},
  {'type': 'ToTensor', 'keys': ['imgs', 'label']}]}

In [6]:
train_dataset = build_dataset(cfg=cfg.data.train)
train_loader = build_dataloader(
        train_dataset,
        videos_per_gpu=8,
        workers_per_gpu=4,
        persistent_workers=False,
        num_gpus=1,
        dist=False)

val_dataset = build_dataset(cfg=cfg.data.val)
val_loader = build_dataloader(
        val_dataset,
        videos_per_gpu=1,
        workers_per_gpu=4,
        persistent_workers=False,
        num_gpus=1,
        dist=False)

## Use MMAction2's Loss

In [7]:
# Best parameters from previous study
best_params = {
    'dropout_ratio': 0.6795542149013333,
    'lr': 7.886714129990479e-06,
    'max_norm': 41,
    'with_pool2': True,
    'bottleneck_mode': 'ir',
    'norm_eval': False,
    'bn_frozen': False,
    'alpha': 1.0 
}

In [8]:
# Set random seed for reproducibility
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True  # Ensures deterministic behavior (might slow down training)
torch.backends.cudnn.benchmark = False


# Configure logging
logging.basicConfig(filename='training_adam_mixup_min_loss.log', 
                    filemode='w', 
                    format='%(asctime)s - %(levelname)s - %(message)s', 
                    level=logging.INFO)

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set hyperparameters directly
dropout_ratio = best_params['dropout_ratio']
lr = best_params['lr']
max_norm = best_params['max_norm']
alpha = best_params['alpha']

# Backbone parameters
cfg.model.backbone.with_pool2 = best_params['with_pool2']
cfg.model.backbone.bottleneck_mode = best_params['bottleneck_mode']
cfg.model.backbone.norm_eval = best_params['norm_eval']
cfg.model.backbone.bn_frozen = best_params['bn_frozen']

# Fixed pretrained URL
cfg.model.backbone.pretrained = 'https://download.openmmlab.com/mmaction/recognition/csn/ircsn_from_scratch_r50_ig65m_20210617-ce545a37.pth'

# Adjust config parameters
cfg.model.cls_head.dropout_ratio = dropout_ratio

# Initialize model, criterion, optimizer, scheduler
model = build_model(cfg.model, train_cfg=None, test_cfg=cfg.get('test_cfg')).to(device)
# model.cls_head.loss_cls = nn.BCEWithLogitsLoss()

# Use Adam optimizer
optimizer = optim.Adam(
    model.parameters(),
    lr=lr,
    weight_decay=0.00001
)

# Early stopping parameters
total_epochs = 60
eval_interval = 1
patience = 5
best_val_loss = float("inf")
epochs_without_improvement = 5

# Mixup Blending instance
mixup = MixupBlending(num_classes=cfg.model.cls_head.num_classes, alpha=alpha)


# Initialize lists to store losses and accuracy
training_log = []


# Validation loop with accuracy and F1 score calculation
for epoch in range(total_epochs):

    # Training loop
    model.train()
    total_train_loss = 0

    for _, data in enumerate(train_loader):
        inputs, labels = data['imgs'].to(device), data['label'].to(device)

        # Convert labels to one-hot encoding
        labels_one_hot = F.one_hot(labels, num_classes=cfg.model.cls_head.num_classes).float()

        # Apply Mixup
        mixed_inputs, mixed_labels = mixup.do_blending(inputs, labels_one_hot)

        optimizer.zero_grad()
        outputs = model(mixed_inputs, mixed_labels, return_loss=True)
        loss = outputs['loss_cls']

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)
        optimizer.step()

        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)
    print(f'Epoch [{epoch + 1}/{total_epochs}], Train Loss: {avg_train_loss:.4f}')

    # Validation loop (every `eval_interval` epochs)
    if (epoch + 1) % eval_interval == 0:
        model.eval()
        total_val_loss = 0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for val_data in val_loader:
                val_inputs, val_labels = val_data['imgs'].to(device), val_data['label'].to(device)

                val_results = model(val_inputs, return_loss=False)
                val_loss = model(val_inputs, val_labels, return_loss=True)['loss_cls']

                total_val_loss += val_loss.item()

                # Collect predictions and true labels
                predictions = np.argmax(val_results, axis=1)
                true_labels = val_labels.cpu().numpy()

                all_preds.extend(predictions)
                all_labels.extend(true_labels)

        avg_val_loss = total_val_loss / len(val_loader)
        accuracy = accuracy_score(all_labels, all_preds)
        avg_f1 = weighted_f1_score(all_labels, all_preds)

        print(f'Epoch {epoch + 1}, Validation Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.4f}, Average F1 Score: {avg_f1:.4f}')

        # Check if validation loss improved
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_without_improvement = 0  # Reset counter
            print(f'Epoch {epoch + 1}, New best validation loss: {avg_val_loss:.4f}')
        else:
            epochs_without_improvement += 1

        # Early stopping check
        if epochs_without_improvement >= patience:
            print(f'Early stopping at epoch {epoch + 1} due to no improvement in validation loss.')
            break

        # Log results for this epoch
        training_log.append({
            'epoch': epoch + 1,
            'train_loss': avg_train_loss,
            'val_loss': avg_val_loss,
            'accuracy': accuracy,
            'avg_f1': avg_f1,
            'type': 'baseline'
        })

# Convert the log to a DataFrame
df = pd.DataFrame(training_log)

# Write the log to a CSV file
csv_file = 'training_log.csv'
df.to_csv(csv_file, index=False)

print(f"Training complete. Best validation loss: {best_val_loss:.4f}. Log saved to {csv_file}.")

2024-12-12 13:07:28,317 - mmaction - INFO - load model from: https://download.openmmlab.com/mmaction/recognition/csn/ircsn_from_scratch_r50_ig65m_20210617-ce545a37.pth
2024-12-12 13:07:28,317 - mmaction - INFO - load checkpoint from http path: https://download.openmmlab.com/mmaction/recognition/csn/ircsn_from_scratch_r50_ig65m_20210617-ce545a37.pth


Epoch [1/60], Train Loss: 3.9237
Epoch 1, Validation Loss: 3.7029, Accuracy: 0.2614, Average F1 Score: 0.2081
Epoch 1, New best validation loss: 3.7029
Epoch [2/60], Train Loss: 3.7242
Epoch 2, Validation Loss: 3.1874, Accuracy: 0.3948, Average F1 Score: 0.3246
Epoch 2, New best validation loss: 3.1874
Epoch [3/60], Train Loss: 3.5424
Epoch 3, Validation Loss: 2.8146, Accuracy: 0.4830, Average F1 Score: 0.4098
Epoch 3, New best validation loss: 2.8146
Epoch [4/60], Train Loss: 3.3846
Epoch 4, Validation Loss: 2.5024, Accuracy: 0.5000, Average F1 Score: 0.4237
Epoch 4, New best validation loss: 2.5024
Epoch [5/60], Train Loss: 3.2544
Epoch 5, Validation Loss: 2.2778, Accuracy: 0.5425, Average F1 Score: 0.4722
Epoch 5, New best validation loss: 2.2778
Epoch [6/60], Train Loss: 3.1196
Epoch 6, Validation Loss: 2.0664, Accuracy: 0.5719, Average F1 Score: 0.5123
Epoch 6, New best validation loss: 2.0664
Epoch [7/60], Train Loss: 2.9899
Epoch 7, Validation Loss: 1.8395, Accuracy: 0.6000, Ave